In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import re
from nltk.corpus import stopwords
# Read the Data 
data = pd.read_csv("/home/administrator/edureka-course/Edureka-course-assignments-questionbank/3rd-minproject-twitter-sentiments/dataset_591_7iqowf3/Dataset/Twitter_Data.csv")

# Change our dependent variable to categorical
data['category'] = data['category'].map({0: 'Neutral', -1: 'Negative', 1: 'Positive'})

# Drop all null/missing values
data.dropna(inplace=True)

# Text cleaning 
def clean_text(text):
    # Remove every symbol except alphanumeric
    text = re.sub(r'\W', ' ', text)
    # Transform all words to lower case
    text = text.lower()
    # Remove punctuation and stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Apply text cleaning to the 'Tweets' column and create 'cleaned_tweets' column
data['cleaned_tweets'] = data['clean_text'].apply(clean_text)

# Create one-hot encoding for each sentence
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data['cleaned_tweets'])
X = tokenizer.texts_to_sequences(data['cleaned_tweets'])


# Add padding from the front side
max_len = max([len(x) for x in X])
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_len, padding='pre')

# Build LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 100, input_length=max_len),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Do dummy variable creation for the dependent variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['category'])

# Split the data into tests and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train new model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Normalize the prediction
predictions = model.predict(X_test)
normalized_predictions = np.argmax(predictions, axis=1)

# Measure performance metrics and accuracy
print(classification_report(y_test, normalized_predictions))

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: initialization failed